<a href="https://colab.research.google.com/github/hholl19/Deep-Leaning-Class-Notes/blob/hholl19_added_things/Applied_Deep_Learning_2nd_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
import pandas as pd
import numpy as np
import os
import pathlib 
from pathlib import Path
#import cv2
#from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dropout, Dense, Flatten, Conv2D, MaxPooling2D 
from keras.layers.normalization import BatchNormalization 
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg19 import VGG19

from numpy import expand_dims
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from matplotlib import pyplot

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
file_path= '/content/drive/MyDrive/Kaggle-Applied Deep Learning Project/Inputs'
folders = os.listdir(file_path)
print(folders)

['test.csv', 'train.csv', 'train', 'test']


In [4]:
df_train= pd.read_csv('/content/drive/MyDrive/Kaggle-Applied Deep Learning Project/Inputs/train.csv')
df_train.head()

,image_id,breed
0,a8b3ad1dde,nerodia-erythrogaster
1,8b492b973d,pantherophis-vulpinus
2,929b99ea92,thamnophis-sirtalis
3,bbac7385e2,pantherophis-obsoletus
4,ef776b1488,agkistrodon-contortrix


In [5]:
df_train["image_id"]="/content/drive/MyDrive/Kaggle-Applied Deep Learning Project/Inputs/train/" + df_train["image_id"] + ".jpg"
df_train.head()

,image_id,breed
0,/content/drive/MyDrive/Kaggle-Applied Deep Lea...,nerodia-erythrogaster
1,/content/drive/MyDrive/Kaggle-Applied Deep Lea...,pantherophis-vulpinus
2,/content/drive/MyDrive/Kaggle-Applied Deep Lea...,thamnophis-sirtalis
3,/content/drive/MyDrive/Kaggle-Applied Deep Lea...,pantherophis-obsoletus
4,/content/drive/MyDrive/Kaggle-Applied Deep Lea...,agkistrodon-contortrix


In [6]:
vgg19 = VGG19(include_top=False,
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)

In [7]:
for layer in vgg19.layers:
    layer.trainable = False

In [8]:
# Instantiate the sequential model and add the VGG19 model: 
model = Sequential()
model.add(vgg19)

# Add the custom layers atop the VGG19 model: 
model.add(Flatten(name='flattened'))
model.add(Dropout(0.3, name='dropout'))
model.add(Dense(35, activation='softmax', name='predictions'))

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
train_datagen = ImageDataGenerator(rescale=1/255,
        validation_split=0.2,
        data_format='channels_last',
        rotation_range=30,
        horizontal_flip=True,
        fill_mode='reflect') 
test_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

In [11]:
train_gen = train_datagen.flow_from_dataframe(
        dataframe=df_train,
        directory="/content/drive/MyDrive/Kaggle-Applied Deep Learning Project/Inputs/train/",
        x_col="image_id",
        y_col="breed",
        subset="training",
        batch_size=32,
        seed=123,
        shuffle=True,
        color_mode="rgb",
        class_mode="categorical", # this does the work of encoding
        target_size=(224,224)) # Change acc to model requirements

Found 4407 validated image filenames belonging to 35 classes.


In [12]:
test_gen = train_datagen.flow_from_dataframe(
        dataframe=df_train,
        directory="/content/drive/MyDrive/Kaggle-Applied Deep Learning Project/Inputs/train/",
        x_col="image_id",
        y_col="breed",
        subset="validation",
        batch_size=32,
        seed=123,
        shuffle=True,
        color_mode="rgb",
        class_mode="categorical", # this does the work of encoding
        target_size=(224,224))

Found 1101 validated image filenames belonging to 35 classes.


In [13]:
model.fit(train_gen, steps_per_epoch=13, 
                    epochs=65, validation_data=test_gen, 
                    validation_steps=13)

Epoch 1/65
13/13 [==============================] - 35s 1s/step - loss: 5.2878 - accuracy: 0.0523 - val_loss: 4.7406 - val_accuracy: 0.0457
Epoch 2/65
13/13 [==============================] - 12s 931ms/step - loss: 4.7128 - accuracy: 0.0773 - val_loss: 4.2631 - val_accuracy: 0.0697
Epoch 3/65
13/13 [==============================] - 11s 915ms/step - loss: 4.2158 - accuracy: 0.0969 - val_loss: 3.8889 - val_accuracy: 0.0913
Epoch 4/65
13/13 [==============================] - 16s 1s/step - loss: 3.7600 - accuracy: 0.0982 - val_loss: 3.8886 - val_accuracy: 0.0865
Epoch 5/65
13/13 [==============================] - 11s 878ms/step - loss: 3.6867 - accuracy: 0.1206 - val_loss: 3.9271 - val_accuracy: 0.1034
Epoch 6/65
13/13 [==============================] - 11s 867ms/step - loss: 3.8974 - accuracy: 0.1307 - val_loss: 3.8178 - val_accuracy: 0.1130
Epoch 7/65
13/13 [==============================] - 11s 871ms/step - loss: 3.8206 - accuracy: 0.1476 - val_loss: 3.7963 - val_accuracy: 0.1106
Epoch

In [14]:
#model.fit(train_gen, steps_per_epoch=15, 
                    #epochs=16, validation_data=test_gen, 
                    #validation_steps=15)